<a href="https://colab.research.google.com/github/kimmiegeorge/CreditCards/blob/main/IdentifyPersistentCardQuarters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Determine Persistent Cards**

Goal: Determine cards with persistent presence, allowing for flexibility in "persistence"

In a given month, determine how many of previous fiscal quarters card was present for, and determine sample construction based on quarters of deposit presence

In [1]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table
import pandas as pd
import numpy as np
import gc

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

project = 'omri-09212023' # Project ID inserted based on the query results selected to explore
location = 'us-central1' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

In [2]:
# Running this code will display the query used to generate your previous job
job = client.get_job('bquxjob_7ebfa9d1_18b6d8bc51e') # Job ID inserted based on the query results selected to explore
#print(job.query)
results = job.to_dataframe()

**Load Data**


1.   BiqQuery results
2.   Create month ID DF
3.   Load EA data and create EA month sequence



In [3]:
# create month id data frame
months = results[['year', 'month']].drop_duplicates()
months = months.sort_values(by=['year', 'month'])
months.reset_index(inplace = True)
months['MonthID'] = months.index
del months['index']

In [4]:
# load EA file
ea_data = pd.read_csv("/content/SampleEAData.csv")
ea_data = ea_data[['TICKER', 'datadate']].copy()
ea_data['datadate'] = pd.to_datetime(ea_data['datadate'])
ea_data['datadate_year'] = ea_data['datadate'].dt.year
ea_data['datadate_month'] = ea_data['datadate'].dt.month
ea_data.sort_values(by = ['TICKER', 'datadate'])
ea_data.reset_index(inplace = True)
ea_data['EA_ID'] = ea_data.index

In [5]:
# filter results file to only keep those with ticker in EA file
results = results[results['MERCHANT_TICKER'].isin(ea_data['TICKER'].unique())].copy()

Expand ea data file to include monthly observations

In [6]:
# merge to get month id
ea_data = pd.merge(ea_data, months, left_on = ['datadate_year', 'datadate_month'],
                      right_on = ['year', 'month'], how = 'left')
# drop missing
ea_data = ea_data[~ea_data['MonthID'].isna()]
# start MonthID
ea_data['StartMonthID'] = ea_data['MonthID'] - 2

In [7]:
# FUNCTIONS
# function to get values between start ID and end ID
def get_values_between(start_id, end_id):
  start_id = int(start_id)
  end_id = int(end_id)
  values = []
  for i in range(start_id, end_id + 1):
    values.append(i)
  return values

def expand_data_frame(original_df):
  """Expands a Pandas DataFrame to include an observation for each firm and ea_id for each value between startID and endID.

  Args:
    original_df: The original DataFrame (Pandas DataFrame).

  Returns:
    An expanded DataFrame (Pandas DataFrame).
  """

  new_df = pd.DataFrame()

  for index, row in original_df.iterrows():
    firm = row['TICKER']
    ea_id = row['EA_ID']
    start_id = row['StartMonthID']
    end_id = row['MonthID']

    values = get_values_between(start_id, end_id)

    for value in values:
      new_df = new_df.append({
        'TICKER': firm,
        'EA_ID': ea_id,
        'MonthID': value
      }, ignore_index=True)

  return new_df


In [8]:
expanded_ea = expand_data_frame(ea_data)

**Determine Card-Firm-Quarters**

Loop through groups of cards, and determine unique card-firm-quarter observations

Append card-firm-quarter observations that meet requirements to a set, which will then be merged with expanded ea to create card-firm-month-quarter observations to send back to BQ

In [12]:
# misc functions
def generate_chunks(list, chunk_size):
  for i in range(0, len(list), chunk_size):
    yield list[i:i + chunk_size]

def getLags(df):

  df['Lag1'] = df.groupby(['ARM_ID_NBR', 'MERCHANT_TICKER'])['EA_ID'].shift(1)
  df['Lag2'] = df.groupby(['ARM_ID_NBR', 'MERCHANT_TICKER'])['EA_ID'].shift(2)
  df['Lag3'] = df.groupby(['ARM_ID_NBR', 'MERCHANT_TICKER'])['EA_ID'].shift(3)
  df['Lag4'] = df.groupby(['ARM_ID_NBR', 'MERCHANT_TICKER'])['EA_ID'].shift(4)

  # differences
  df['Diff1'] = df['EA_ID'] - df['Lag1']
  df['Diff2'] = df['EA_ID'] - df['Lag2']
  df['Diff3'] = df['EA_ID'] - df['Lag3']
  df['Diff4'] = df['EA_ID'] - df['Lag4']

  df['OK2'] = np.where(df['Diff1'].eq(1) & df['Diff2'].eq(2), True, False)
  df['OK3'] = np.where(df['Diff1'].eq(1) & df['Diff2'].eq(2) & df['Diff3'].eq(3) , True, False)
  df['OK4'] = np.where(df['Diff1'].eq(1) & df['Diff2'].eq(2) & df['Diff3'].eq(3) & df['Diff4'].eq(4), True, False)

  df.drop(columns=['Lag1', 'Lag2', 'Lag3', 'Lag4',
                   'Diff1', 'Diff2', 'Diff3', 'Diff4'], inplace=True)

  return(df)

In [16]:
# TESTER
# full count
chunk_size = 500
cards = results.ARM_ID_NBR.unique()
count = 0
num_chunks = round(len(cards)/chunk_size)

# loop through chunks
for chunk in generate_chunks(cards, chunk_size):

  # pull results
  sub_results = results[results['ARM_ID_NBR'].isin(chunk)]

  if count == 0:
    break

In [29]:
sub_results['YearMonth'] = sub_results['year'].astype(str) + '_' + sub_results['month'].astype(str)

In [31]:
months['YearMonth'] = months['year'].astype(str) + '_' + months['month'].astype(str)

In [34]:
expanded_ea.head()

,TICKER,EA_ID,MonthID
0,MSFT,2,0
1,MSFT,2,1
2,MSFT,2,2
3,MSFT,3,3
4,MSFT,3,4


In [49]:
def FilterObs(results, chunk_size):

  # csv files
  twoQuartersFile = "/content/TwoQuarterFile.csv"
  threeQuartersFile = "/content/ThreeQuarterFile.csv"
  fourQuartersFile = "/content/FourQuarterFile.csv"

  # counts
  twoCount = 0
  threeCount = 0
  fourCount = 0

  # pull cards
  cards = results.ARM_ID_NBR.unique()

  # full count
  count = 0
  num_chunks = round(len(cards)/chunk_size)

  # loop through chunks
  for chunk in generate_chunks(cards, chunk_size):

    count += 1
    if count % 100 == 0:
      print("On chunk " +  str(count) + " out of " + str(num_chunks) )

    # pull results
    sub_results = results[results['ARM_ID_NBR'].isin(chunk)]

    # merge to get month id
    sub_results['YearMonth'] = sub_results['year'].astype(str) + '_' + sub_results['month'].astype(str)
    months['YearMonth'] = months['year'].astype(str) + '_' + months['month'].astype(str)
    sub_results = pd.merge(sub_results, months, on = ['YearMonth'], how = 'left')

    # merge to get EAID
    sub_results = pd.merge(sub_results, expanded_ea,
                           left_on = ['MERCHANT_TICKER', 'MonthID'],
                           right_on = ['TICKER', 'MonthID'], how = 'left')

    # only keep individual quarterly observations
    sub_results = sub_results[['ARM_ID_NBR', 'MERCHANT_TICKER', 'EA_ID']].drop_duplicates().copy()

    # sort values
    sub_results = sub_results.sort_values(by = ['ARM_ID_NBR', 'MERCHANT_TICKER', 'EA_ID'])

    # lags
    lags = getLags(sub_results)

    ### 2 previous quarters
    twoPrev = lags[lags['OK2'] == True]
    twoPrev = twoPrev[['ARM_ID_NBR', 'MERCHANT_TICKER', 'EA_ID']]

    if twoCount == 0:
      twoPrev.to_csv(twoQuartersFile)
      twoCount += 1
    else:
      twoPrev.to_csv(twoQuartersFile, method = 'a', headers = False)

    del twoPrev

    ### 3 previous quarters
    threePrev = lags[lags['OK3'] == True]
    threePrev = threePrev[['ARM_ID_NBR', 'MERCHANT_TICKER', 'EA_ID']]

    if threeCount == 0:
      threePrev.to_csv(threeQuartersFile)
      threeCount += 1
    else:
      threePrev.to_csv(threeQuartersFile, method = 'a', headers = False)

    del threePrev

    ### 4 previous quarters
    fourPrev = lags[lags['OK4'] == True]
    fourPrev = fourPrev[['ARM_ID_NBR', 'MERCHANT_TICKER', 'EA_ID']]

    if fourCount == 0:
      fourPrev.to_csv(fourQuartersFile)
      fourCount += 1
    else:
      fourPrev.to_csv(fourQuartersFile, method = 'a', headers = False)

    del fourPrev

    gc.collect()

    if count == 1:
      break




In [50]:
FilterObs(results, 500)